## import requests
import random

In [2]:
def get_station(header, start_page=1, end_page=2, row_nmb=10):
    """
    return a list of dictionaries that contains charging station info
    """
    curr_page = start_page #specify starting page to scrape, usually it is page 1.
    page_end = end_page  #specify the page wanted to end scraping. Don't worry if number is too large, 
                                                                                        #we ends when page is empty.
    row_nmb = row_nmb  #specify how many rows per request returns.
    station_info = []  #summarizing all the station location data.
    
    #Starting iteration going over website to scrape station locations
    while curr_page <= page_end:
        REQUEST_URL = 'https://www.teld.cn/StationNetwork/GetStationNetword?ProvinceName=&CityName=&KeyWords=&\
RegionName=&type=&page={}'.format(curr_page) + '&rows={}'.format(row_nmb)
        resp = requests.get(REQUEST_URL, headers=header)
        curr_page += 1
        json_requestData = resp.json()
        station_info = station_info + json_requestData['rows']
    return station_info

In [3]:
def get_piles(station_code, proxies_pool, header, time_out=20):
    """
    return json object containing piles information
    "pile code" format - string, station_info[0]['code']
    """
    STATION_URL = 'https://www.teld.cn/StationNetwork/GetChargingStationByCodeList?StationNo='+ station_code
    proxy_idx = random.randint(0,len(proxies_pool)-1)
#     print(proxy_idx)
#     try:
    resp = requests.get(STATION_URL, proxies=proxies_pool[proxy_idx], headers=header, timeout=time_out)
#     except Exception, e:
      
    pile_info = resp.json()
    return pile_info

In [99]:
#-----GET ALL THE STATONS IN ONE LIST-----#
def get_allStations(hdr):
    """
    Input:
        hdr - dictionary, detailed spec for headers
    Output:
        stations_list - list, containing all stations info from website, each in format of dict
    Each station has seven attribute: isInterconnection, name, address, 
                                isFast, stationType, lat, lng, id
    """
    REQUEST_URL = 'http://baseapi.teld.cn/api/GetAllStations/StaV3?coordinateType=gaode'
    stations_resp = requests.get(REQUEST_URL, headers=hdr)
    json_requestData = stations_resp.json()
    stations_list = json_requestData['data']['stations']
    return stations_list

In [131]:
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/\
59.0.3071.115 Safari/537.36', 
       'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
REQUEST_URL = 'http://baseapi.teld.cn/api/GetAllStations/StaV3?coordinateType=gaode'
# files = {'file': open('teld_app_test.txt', 'rb')}
stations_resp = requests.get(REQUEST_URL, headers=hdr)
json_requestData = stations_resp.json()

In [136]:
for i in json_requestData['data']['stations'][1].keys():
    print i, '     ', json_requestData['data']['stations'][1][i]

isInterconnection       N
name       北京经开荣京道充电站
address       北京市市辖区大兴区经济技术开发区地盛北街1号地下二层停车场
isFast       Y
stationType       公共站
lat       39.785593
lng       116.497084
id       96854d4a-344f-44a1-bcaf-ae73fa82d114


In [143]:
str(json_requestData['data']['stations'][1]['stationType'].encode('utf-8'))

'\xe5\x85\xac\xe5\x85\xb1\xe7\xab\x99'

In [106]:
# json_requestData['data']['stations'][7000]

In [144]:
#-----GET ONE STATION DETAILS-----#
def get_oneStation_APP(hdr, stationId):
    """
    Input: 
        hdr - dictionary, detailed spec for headers
        stationId - string, each station's own station code
    Output:
        station_detailedInfo - dictionary, containing detailed station information, 37 keys
    """
    REQUEST_URL = 'http://baseapi.teld.cn/api/GetStationDetails/StaV3?coordinateType=gaode\
    &lat=0.000000&lng=0.000000&stationId='+ stationId
    terminal_resp = requests.get(REQUEST_URL, headers=hdr)
    json_requestData = terminal_resp.json()
    station_detailedInfo = json_requestData['data']
    return station_detailedInfo

In [259]:
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/\
59.0.3071.115 Safari/537.36', 
       'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
REQUEST_URL = 'http://baseapi.teld.cn/api/GetStationDetails/StaV3?\
coordinateType=gaode&lat=0.000000&lng=0.000000&\
stationId=c09327a9-fe5b-4b4e-9bff-0e50097000e9'
terminal_resp = requests.get(REQUEST_URL, headers=hdr)
json_requestData = terminal_resp.json()
station_detailedInfo = json_requestData['data']

In [251]:
# for i in station_detailedInfo.keys():
#     print i, ' = str(station_detailedInfo[\'%s\'])' % (i)
# for i in station_detailedInfo.keys():
#     print i+',',
for i in station_detailedInfo.keys():
    print i,'  -----  ', station_detailedInfo[i]
# print str(station_detailedInfo['payType'].encode('utf-8'))
# if station_detailedInfo['specialDesc'] is None :
#     print 1
# else:
#     print 2
# collected  = str(station_detailedInfo['specialDesc'].encode('utf-8'))
# print(collected)

isInterconnection   -----   False
businessHours   -----   周一至周日00:00-24:00
operatorPhone   -----   4001-300-001
outNetworkTerminalNum   -----   0
usingTerminalNum   -----   1
advertisementHtml   -----   
advertisementImg   -----   
positionDesc   -----   靠近江边5号楼地下负一层（只限对业主和访客开放）
serviceScore   -----   3.0
stationAddress   -----   广东省深圳市龙岗区深汕公路与深汕高速交汇处（双龙站B出口转同乐方向）
fastTerminalIdleNum   -----   0
lng   -----   114.302623
parkFee   -----   
specialDesc   -----   
isTeldPersonalTerminal   -----   True
fastTerminalNum   -----   0
specialDescV3   -----   
operatorName   -----   深圳特来电新能源有限公司
score   -----   3.8
slowTerminalIdleNum   -----   1
collected   -----   N
priceScore   -----   3.6
environmentScore   -----   3.7
tags   -----   [{u'tagLabel': u'\u81ea\u8425', u'tagColor': u'00a6ca'}]
payType   -----   本APP，特来电微信公众号
defaultImg   -----   https://resource.teld.cn/teldimage/115/57be2b78b95f4e24941c68c5e190a3b4.jpg
priceDesc   -----   电费0.2794-1.0759元/度,服务费0.7000元/度
hasLock   -----   N
stat

In [243]:
print(json_requestData['data']['stationId'])

c09327a9-fe5b-4b4e-9bff-0e50097000e9


In [139]:
#------GET PILES ID FROM TERMINAL-----#
# hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/\
# 59.0.3071.115 Safari/537.36', 
#        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
# #        'Host':'baseapi.teld.cn',
# #        'Content-Type':'application/x-www-form-urlencoded',
# #        'Accept':'*/*',
# #        'Connection':'keep-alive',
# #        'Device':'app_version=3.3.0&os_version=10.3.3&client=iso&device_name=iphone 6s',
# #        'User-Agent':'Teld/3.3.0 (iPhone; ios 10.3.3; Scale/2.00)',
# #        'Accept-Language':'en-US;q=1, zh-Hans-US;q=0.9, ko-KR;q=0.8, zh-Hant-US;q=0.7',
# #        'Content-Length':'64',
# #        'Accept-Encoding':'gzip,deflate'}
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetTerminalOfStation/StaV3?keyword=&orderBy=&stationId=045725c3-1b23-4108-837d-77dfb6b53e7d'
# pile_resp = requests.post(REQUEST_URL, headers=hdr)
# json_requestData = pile_resp.json()

In [97]:
#-----GET ONE CHARGING PILE DETAILS FROM TERMINAL-----#
def get_onePile_APP(header, pileCode):
    """
    Input:
        hdr - dictionary, detailed spec for headers
        pileCode - string, each charging pile has its own pile code
    Output:
        pile_detailedInfo - dictionary, containing detailed station information, 49 keys
    """
    REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pileCode
    pile_resp = requests.get(REQUEST_URL, headers=hdr)
    json_requestData = pile_resp.json()
    pile_detailedInfo = json_requestData['data']['terminal']
    return pile_detailedInfo

In [319]:
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/\
59.0.3071.115 Safari/537.36', 
       'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?\
pillCode=4403070007101'
pile_resp = requests.get(REQUEST_URL, headers=hdr)
json_requestData = pile_resp.json()

In [320]:
for i in json_requestData['data']['terminal'].keys():
    print i, '  -----  ', json_requestData['data']['terminal'][i]

interconnectionId   -----   
isInterconnection   -----   N
allowEconomyCharge   -----   False
isMyPersonalTerminal   -----   False
stateName   -----   空闲
pillCode   -----   4403070007101
tag   -----   国标2011 60kW
voltage   -----   200V-500V
parkFee   -----   免费（以场站方公告为准）
OpstateName   -----   运营中
pillID   -----   f7e89e19-4b62-4537-96fd-6c06488fd2dc
Description   -----   00:00~24:00:0.700000:00~24:00:1.0500
specialDesc   -----   
notOpenOutside   -----   N
isMoneyNotEnough   -----   False
allowLimitCharge   -----   False
specialDescV3   -----   
OpState   -----   3
isTeldPersonalTerminal   -----   True
isFast   -----   Y
carPark   -----   无
pillName   -----   101号直流
allowOrderCharge   -----   False
allowCharge   -----   True
canChargeMoney   -----   0
orderTime   -----   
ChargingPort   -----   国标2011 
power   -----   15kW-60kW
stateNameV3   -----   空闲
chargeApplyTime   -----   95
linkPhone   -----   4001300001
payType   -----   本APP，特来电微信公众号
priceDesc   -----   电费:00:00~24:00:1.0500;服

In [240]:
len(json_requestData['data']['terminal']['linkPhone'])

10

In [241]:
print(json_requestData['data']['terminal']['staID'])

c09327a9-fe5b-4b4e-9bff-0e50097000e9


In [121]:
print(json_requestData['data']['terminal']['pillID'])

0497f693-096b-499d-9969-d5b0326084c7


In [316]:
hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/\
59.0.3071.115 Safari/537.36', 
       'Accept':'*/*',
       'User-Agent':'Teld/3.3.0(iPhone;iOS 10.3.3;Scale/2.00)',
       'Cookie':'TELDSID=5a27131b-9d4f-453d-be37-aba843162dfe'}
REQUEST_URL = 'http://baseapi.teld.cn/api/GetChargeListBySta/StaV3?itemNumPerPage=200&\
pageNum=1&staID=d4035b02-5f27-4ae9-b0b1-dd10010f5b7e'
pile_resp = requests.get(REQUEST_URL, headers=hdr)
json_requestData = pile_resp.json()
chargelist = json_requestData['data']['items']

In [317]:
chargelist[1].keys()

[u'PileName',
 u'hasCertificate',
 u'carTypeName',
 u'userNickName',
 u'chargeTime']

In [318]:
for i in chargelist:
    print i['PileName'],'----', i['userNickName'],'----',i['chargeTime'], '-----', i['carTypeName']
    
print(str(chargelist[2]['userNickName']) )
print(str(chargelist[3]['userNickName'].encode('utf-8')) )

str(chargelist[2]['userNickName']) == str(chargelist[3]['userNickName'])

103号 ---- 1***3 ---- 9小时前 ----- 未知车型
102号 ---- Y***O ---- 13小时前 ----- 福汽新龙马启腾M70EV
105号 ---- 丁***丁 ---- 13小时前 ----- 众泰E200
106号 ---- 土***豆 ---- 14小时前 ----- 未知车型
101号 ---- 你***好 ---- 14小时前 ----- 未知车型
102号 ---- 萧***平 ---- 14小时前 ----- 未知车型
103号 ---- 1***4 ---- 14小时前 ----- 未知车型
102号 ---- 萧***平 ---- 15小时前 ----- 未知车型
101号 ---- 林***生 ---- 15小时前 ----- 福汽新龙马启腾M70EV
103号 ---- 旺***仔 ---- 15小时前 ----- 长安逸动ev
104号 ---- 1***3 ---- 17小时前 ----- 未知车型
101号 ---- 1***8 ---- 19小时前 ----- 未知车型
102号 ---- 一***安 ---- 19小时前 ----- 未知车型
103号 ---- Y***O ---- 19小时前 ----- 未知车型
102号 ---- Y***O ---- 19小时前 ----- 未知车型
106号 ---- 青***人 ---- 20小时前 ----- 奇瑞EQ-2015
103号 ---- 天***道 ---- 08-15 07:18 ----- 未知车型
105号 ---- 1***3 ---- 08-15 02:27 ----- 未知车型
103号 ---- 要***电 ---- 08-15 00:34 ----- 江淮iEV4
102号 ---- 朱*** ---- 08-14 23:00 ----- 未知车型
104号 ---- 1***6 ---- 08-14 22:50 ----- 福汽新龙马启腾M70EV
103号 ---- 陈***浩 ---- 08-14 22:09 ----- 福汽新龙马启腾M70EV
105号 ---- 1***3 ---- 08-14 20:26 ----- 未知车型
102号 ---- 1***7 ---- 08-14 20:14 ----- 未知车型

UnicodeEncodeError: 'ascii' codec can't encode character u'\u4e01' in position 0: ordinal not in range(128)

In [294]:
print(chargelist[7]['chargeTime'])
print(chargelist[2]['chargeTime'])

14小时前
1小时前
